In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Image titles
image_titles = ['Autistic', 'Non-Autistic']
# Assuming you have 'image_titles', 'test_data', and 'class_indices' arrays defined

num_images = len(image_titles)

# Convert one-hot encoded labels to integer labels
test_labels_int = np.argmax(test_labels, axis=1)
# Find the indices of the first image from each class
class_indices = [np.where(test_labels_int == i)[0][0] for i in range(len(image_titles))]

# Create an array to store the images
image_array = []

# Create subplots
fig, ax = plt.subplots(1, num_images, figsize=(15, 5))

for i, title in enumerate(image_titles):
    ax[i].set_title(title, fontsize=12)

    # Display the image from test data
    img = test_data[class_indices[i]]
    image_array.append(img)  # Store the image in the array

    ax[i].imshow(img)
    ax[i].axis('off')

plt.tight_layout()
plt.show()

X = base_preprocess(np.array(image_array))

In [ ]:
##smoothGrad
%%time
from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
# Generate saliency map with smoothing that reduce noise by adding noise
saliency_map = saliency(score,
                        X,
                        smooth_samples=90, # The number of calculating gradients iterations.
                        smooth_noise=0.90) # noise spread level.

## Since v0.6.0, calling normalize() is NOT necessary.
# saliency_map = normalize(saliency_map)

# Render
f, ax = plt.subplots(nrows=1, ncols=7, figsize=(12, 4))
for i, title in enumerate(image_titles):
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(saliency_map[i], cmap='jet')
    ax[i].axis('off')
plt.tight_layout()
#plt.savefig('images/smoothgrad.png')
plt.show()

In [ ]:
##scoreCam
%%time
from matplotlib import cm
from tf_keras_vis.scorecam import Scorecam

# Create ScoreCAM object
scorecam = Scorecam(model, model_modifier=replace2linear)

# Generate heatmap with Faster-ScoreCAM
cam = scorecam(score,
               X,
               penultimate_layer=-1,
               max_N=10)


# Render
f, ax = plt.subplots(nrows=1, ncols=7, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=12)
    ax[i].imshow(image_array[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5)
    ax[i].axis('off')
plt.tight_layout()
plt.show()